In [226]:
!pip install selenium webdriver_manager BeautifulSoup4
!pip install chromedriver_autoinstaller requests mysql-connector-python

In [227]:
import re
import time
import mysql.connector
from selenium import webdriver
import chromedriver_autoinstaller
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from datetime import datetime, timedelta

In [228]:
chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument('--headless')
chromedriver_autoinstaller.install()

driver = webdriver.Chrome(options=chrome_options)

In [229]:
db_config = {
	'host': 'localhost',
	'user': 'root',
	'password': '1234',
	'database': 'news_sentiment_db'
}

def connect_db():
		return mysql.connector.connect(**db_config)

connect_db()

In [230]:
KEYWORDS = ["SKT", "해킹", "유심"]
PUBLISHER = ["연합뉴스", "뉴시스", "서울경제", "데일리안", "뉴스1", "디지털데일리", "한국경제", "파이낸셜뉴스"]

In [ ]:
def save_articles_to_db(articles):
	conn = connect_db()
	cursor = conn.cursor()
	sql = """
		INSERT INTO news_articles (title, summary, publisher, publish_date, url, comment_count, collected_at)
		VALUES (%s, %s, %s, %s, %s, %s, NOW())
	"""
	cursor.executemany(sql, [(article['title'], article['summary'], article['publisher'],
							  article['publish_date'], article['url'], article['comment_count'])
							 for article in articles])
	conn.commit()
	conn.close()

def save_comments_to_db(article_id, comments):
    conn = connect_db()
    cursor = conn.cursor()
    sql = """
        INSERT INTO news_comments (article_id, parent_comment_id, nickname, content, like_count, dislike_count, comment_date, collected_at)
        VALUES (%s, %s, %s, %s, %s, %s, %s, NOW())
    """
    cursor.executemany(sql, [(article_id, *c) for c in comments])
    conn.commit()
    conn.close()

In [ ]:
# 기사 목록 더보기 클릭
def click_more_article():
	while True:  
		try:
			driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
			time.sleep(5)
			more_btn = driver.find_element(By.CSS_SELECTOR, '#newsct > div.section_latest > div > div.section_more > a')
			more_btn.click()
			time.sleep(1.5)
		except:
			break

# 기사 내 댓글 더보기 클릭
def click_to_comment():
	try:
		driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
		time.sleep(5)
		more_button = driver.find_element(By.CSS_SELECTOR, "a.u_cbox_btn_view_comment")
		more_button.click()
		time.sleep(1.5)

	except:
		print("댓글 더보기 클릭 실패")

# 댓글 모음 내 더보기 클릭
def click_more_comment():
	while True:
		try:
			driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
			time.sleep(5)
			more_btn = driver.find_element(By.CSS_SELECTOR, "a.u_cbox_btn_more")
			more_btn.click()
			time.sleep(1.5)
		except:
			break

In [233]:
# def parse_publish_date(raw_date_str):
#     try:
#         match = re.match(r'(\d+)일전', raw_date_str)
#         if match:
#             days_ago = int(match.group(1))
#             return (datetime.now() - timedelta(days=days_ago)).date()

#         match = re.match(r'(\d{4})\.(\d{2})\.(\d{2})\.', raw_date_str)
#         if match:
#             year, month, day = map(int, match.groups())
#             return datetime(year, month, day).date()

#         print("날짜 형식 미매칭:", raw_date_str)
#         return None

#     except Exception as e:
#         print("날짜 파싱 실패:", raw_date_str, e)
#         return None


In [ ]:
def get_article_data(article, publish_date):
	article_data = {}

	try:
		article_data['title'] = article.find_element(By.CSS_SELECTOR, 'a.sa_text_title > strong.sa_text_strong').text.strip()
		article_data['summary'] = article.find_element(By.CSS_SELECTOR, 'div.sa_text_lede').text.strip()
		article_data['url'] = article.find_element(By.CSS_SELECTOR, 'a.sa_text_title').get_attribute('href')
		article_data['publisher'] = article.find_element(By.CSS_SELECTOR, 'div.sa_text_press').text.strip()
		article_data['publish_date'] = publish_date

		# 기사 날짜 저장 시 사용하려고 했지만, ~일 전으로 뜨는 경우 값이 굳이 계산이 필요 없을 것 같아서 파라미터로 넘기는 날짜를 사용
		# raw_date = article.find_element(By.CSS_SELECTOR, 'div.sa_text_datetime > b').text.strip()
		# article_data['publish_date'] = parse_publish_date(raw_date)

		try:
			comment_element = article.find_element(By.CSS_SELECTOR, 'div.sa_text_info_right > a.sa_text_cmt')
			if comment_element.is_displayed():
				comment_text = comment_element.text.strip()
				comment_number = int(''.join(filter(str.isdigit, comment_text)))
				article_data['comment_count'] = comment_number
			else:
				article_data['comment_count'] = 0
		except:
			article_data['comment_count'] = 0

		return article_data

	except Exception as e:
		print("오류 발생:", e)
		return None


# def get_comment_count(url):
#   try:
#     driver.execute_script("window.open('');")
#     driver.switch_to.window(driver.window_handles[1])
#     driver.get(url)
#     time.sleep(2)
#     soup = BeautifulSoup(driver.page_source, 'html.parser')
#     comment_tag = soup.select_one('span.u_cbox_count')
#     driver.close()
#     driver.switch_to.window(driver.window_handles[0])
	
#     return int(comment_tag.text) if comment_tag else 0
#   except:
#     return 0

In [ ]:
def crawl_by_date(date_str):
	search_url = f"https://news.naver.com/breakingnews/section/105/732?date={date_str}"
	driver.get(search_url)
	time.sleep(2)

	click_more_article()

	data_to_save_article = []
	articles = driver.find_elements(By.CSS_SELECTOR, "li.sa_item")

	for art in articles:
		article_data = get_article_data(art, date_str)

		if not article_data:
			continue

		if not any(k in (article_data['title'] + article_data['summary']) for k in KEYWORDS):
			continue
		if article_data['publisher'] not in PUBLISHER:
			continue

		data_to_save_article.append(article_data)

	if data_to_save_article:
		print(f"{len(data_to_save_article)}개의 기사가 필터링되어 저장됩니다.")
		save_articles_to_db(data_to_save_article)

def run_crawler_daily():
  today = datetime.today()
  yesterday = today - timedelta(days=1)
  date_str = yesterday.strftime('%Y%m%d')
  crawl_by_date(date_str)

In [236]:
def main():
	crawl_by_date("20250508")
	# run_crawler_daily()

if __name__ == "__main__":
		main()

35개의 기사가 필터링되어 저장됩니다.
